In [2]:
import find_assets
import pandas as pd

# Use the check_asset method to retrieve information about the assets
asset_info = find_assets.check_all_assets()

# asset_info[0], asset_info[1]

['IRBT', 'IRBO', 'IRT', 'DTW', 'IR', 'DUDE', 'DUG', 'DUK', 'DUKB', 'IQV', 'LASR', 'IRTC', 'ISDX', 'ISEE', 'MSTR', 'SCHC', 'SCHF', 'SCHG', 'SCHH', 'SQM', 'PVH', 'ECVT', 'SYLD', 'STKS', 'STLA', 'IQ', 'STLD', 'GGB', 'MRPLY', 'RWX', 'SBH', 'IPWR', 'SYNH', 'TAIL', 'IPI', 'IHG', 'IHF', 'IHDG', 'FXC', 'FXB', 'FXA', 'TDIV', 'FWRD', 'FNF', 'SCHI', 'MSFT', 'MPWR', 'SLV', 'INTZ', 'FNDX', 'FNCL', 'INTU', 'FN', 'WRBY', 'SBLK', 'IEP', 'MSB', 'IHI', 'XTN', 'RWT', 'RWR', 'ATEC', 'MSA', 'IT', 'KRYS', 'FVRR', 'FVD', 'CNHI', 'PWSC', 'NYT', 'IIGD', 'SJB', 'IFNNY', 'MS', 'FMS', 'FMNB', 'IDT', 'SZKMY', 'OTLY', 'OVCHY', 'ITGR', 'SCSC', 'LBRDA', 'ASEA', 'DUOL', 'MRVL', 'DURA', 'DUSA', 'LBRDK', 'MRVI', 'DV', 'FUTU', 'MRNS', 'LBRDP', 'LBRT', 'MRSK', 'MRNA', 'DVA', 'MRK', 'FXI', 'DVOL', 'MSGS', 'MSI', 'FXL', 'ITOS', 'ITOT', 'MPLX', 'SBSI', 'MPLN', 'SBSW', 'TKHVY', 'MPC', 'MPB', 'AAAU', 'MTCH', 'LCID', 'LCII', 'FMIL', 'FUN', 'MRO', 'UPV', 'FULT', 'FULC', 'SQQQ', 'FUL', 'SQSP', 'FUBO', 'PFH', 'FMB', 'ACGBY', 'DVY'

KeyboardInterrupt: 